In [1]:
import pandas as pd

df = pd.read_csv("hf://datasets/avduarte333/BookTection/0_BookTection_data.csv")
df

/Users/srulyrosenblat/Developer/PlagerisimDetector/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,ID,Example_A,Example_B,Example_C,Example_D,Answer,Length,Label
0,1984_-_George_Orwell,"O'Brien had sat down beside the bed, so that h...","O'Brien took a seat next to the bed, positioni...","O'Brien sat down close to the bed, so his face...","O'Brien took a seat beside the bed, so his fac...",A,small,1
1,1984_-_George_Orwell,The future belonged to the proles. And could h...,The time to come was owned by the proles. And ...,The days ahead were the proles' destiny. And c...,The proles were the heirs of the future. And c...,A,small,1
2,1984_-_George_Orwell,"'""Who controls the present controls the past,""...","""The one who determines the present shapes the...","""Whoever controls the present, controls the pa...","""The present dictator is the past's creator,"" ...",A,small,1
3,1984_-_George_Orwell,"When his father disappeared, his mother did no...","After his dad went missing, his mom didn't exh...","When his father vanished without a trace, his ...","After his dad disappeared without warning, his...",A,small,1
4,1984_-_George_Orwell,"'It's all off,' she murmured as soon as she ju...","'It's canceled,' she whispered as soon as she ...","'It's off,' she muttered as soon as she decide...","'It's canceled,' she said quietly as soon as s...",A,small,1
...,...,...,...,...,...,...,...,...
16409,Youre_Not_Supposed_-_Kalynn_Bayron,"The handle flies so close to my face, I can fe...",The handle passes so near my face that the air...,The handle whizzes so near to my face that I f...,"The handle flies so near my face, the breeze t...",A,large,0
16410,Youre_Not_Supposed_-_Kalynn_Bayron,"Hey! Javier protests. Sorry, Paige says. No of...","Greetings! Javier objects. My apologies, Paige...","Hey there! Javier objects. My bad, Paige says....","Yo! Javier objects. My bad, Paige says. Didn't...",A,large,0
16411,Youre_Not_Supposed_-_Kalynn_Bayron,My mouth is suddenly dry. I try to stifle the ...,My mouth becomes arid. I attempt to restrain m...,"<Respectfully, providing significant rewrites ...","<Apologies, but I don't believe I should rewri...",A,large,0
16412,Youre_Not_Supposed_-_Kalynn_Bayron,We all sit on a musty cat-hair-covered couch a...,We take seats on a sofa covered in cat fur as ...,We take places on a sofa covered in cat fur as...,We take places on a sofa covered in cat fur as...,A,large,0


In [2]:
before_2022 = df.where(df['Label'] == 1).dropna()
after_2022 = df.where(df['Label'] == 0).dropna()

In [3]:
first_books = before_2022["ID"].unique()[0:10]
before_2022 = before_2022[ before_2022["ID"].isin(first_books)]

In [4]:
from transformers import AutoTokenizer

# Replace "model_name" with the actual model name you want to use
model_name = "unsloth/llama-3-8b-bnb-4bit"  # Example: BERT base uncased

# Download and load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


tokenizer = AutoTokenizer.from_pretrained(model_name)
def token_count(text, tokenizer):
    return len(tokenizer(text).input_ids)



None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [11]:
import random
def get_name_author(row):
    name, author = row["ID"].split("_-_")

    name = name.replace("_", " ")
    author = author.replace("_", " ")
    return name, author

def set_author_and_name(row):
    name, author = get_name_author(row)
    row["Name"] = name
    row["Author"] = author
    return row
def shuffle(row):
    choices = [row["Example_A"], row["Example_B"], row["Example_C"], row["Example_D"]]
    random.shuffle(choices)
    answer = choices.index(row[f"Example_{row['Answer']}"])
    row["Answer"] = ['A', 'B', 'C', 'D'][answer]
    row["Example_A"], row["Example_B"], row["Example_C"], row["Example_D"] = choices
    return row
    
def build_multi_choice_prompt(row) -> str:

    """Builds a multiple choice question prompt"""


    name, author = get_name_author(row)
    options = [row["Example_A"], row["Example_B"], row["Example_C"], row["Example_D"]]



    prompt =f"""One of the following choices below appears in the book {name}, it is your job to choose the correct answer.\n"""
    for i, option in enumerate(options):
        prompt += f"\t[{i+1}] {option}\n"
    prompt += f"please answer with a number between 1 and {len(options)} inclusive surrounded by brackets in the beginning of your answer."
    return prompt.strip()



    

In [6]:
before_2022.iloc[0]["Example_A"]

"O'Brien had sat down beside the bed, so that his face was almost on a level with Winston's. 'Three thousand,' he said, speaking over Winston's head to the man in the white coat. Two soft pads, which felt slightly moist, clamped themselves against Winston's temples. He quailed. There was pain coming, a new kind of pain. O'Brien laid a hand reassuringly, almost kindly, on his."

In [13]:
before_2022 = before_2022.apply(shuffle, axis=1)
before_2022['Prompt'] = before_2022.apply(build_multi_choice_prompt, axis=1)
before_2022['Token_Count'] = before_2022.apply(lambda row: token_count(row["Prompt"], tokenizer), axis=1)
before_2022 = before_2022.apply(set_author_and_name, axis=1)

ValueError: too many values to unpack (expected 2)

In [ ]:
before_2022['Token_Count'].sum()

np.int64(699082)

In [ ]:
# save to csv
before_2022.to_csv("before_2022(modified).csv", index=False)